In [2]:
import pandas as pd

# Load the dataset
file_path = '/Users/kelly/Downloads/Motor_Vehicle_Collisions_-_Crashes_20240226.csv'
df = pd.read_csv(file_path)


/var/folders/6h/p0xzr_zs7hxf012d587r5pb80000gn/T/ipykernel_51321/651163044.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [10]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [3]:
# Convert 'CRASH DATE' column to datetime format
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
# Filter the data to include only crashes since July 2012
df_since_july_2012 = df[df['CRASH DATE'] >= '2012-07-01']

# Remove rows where 'CONTRIBUTING FACTOR VEHICLE 1' is 'Unspecified'
df_known_factors = df_since_july_2012[df_since_july_2012['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

# Find the leading cause of crashes
leading_cause = df_known_factors['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().idxmax()
leading_cause_count = df_known_factors['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().max()

# Calculate the proportion of accidents attributable to the leading cause
total_accidents = df_known_factors.shape[0]
proportion = leading_cause_count / total_accidents

leading_cause, leading_cause_count, proportion


('Driver Inattention/Distraction', 413525, 0.30332047997288986)

In [4]:
causes = df_known_factors['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

In [8]:
causes

Driver Inattention/Distraction                           413525
Failure to Yield Right-of-Way                            122769
Following Too Closely                                    110471
Backing Unsafely                                          76509
Other Vehicular                                           64185
Passing or Lane Usage Improper                            57562
Passing Too Closely                                       51464
Turning Improperly                                        51070
Fatigued/Drowsy                                           47396
Unsafe Lane Changing                                      40626
Traffic Control Disregarded                               36673
Driver Inexperience                                       32358
Unsafe Speed                                              28851
Alcohol Involvement                                       22586
Lost Consciousness                                        20476
Reaction to Uninvolved Vehicle          

In [6]:
causes_sum = causes.sum()

In [7]:
causes_sum

1356578

In [9]:
df_since_july_2012['NUMBER OF PERSONS KILLED'] = df_since_july_2012['NUMBER OF PERSONS KILLED'].fillna(0)

# Filter the data for accidents with two or more fatalities
df_two_or_more_fatalities = df_since_july_2012[df_since_july_2012['NUMBER OF PERSONS KILLED'] >= 2]

# Count the number of accidents with two or more fatalities
total_accidents_two_or_more_fatalities = df_two_or_more_fatalities.shape[0]

total_accidents_two_or_more_fatalities

92

In [11]:
df_at_least_one_fatality = df_since_july_2012[df_since_july_2012['NUMBER OF PERSONS KILLED'] >= 1]

# Count the number of accidents with at least one fatality
total_accidents_at_least_one_fatality = df_at_least_one_fatality.shape[0]

# Calculate the total number of accidents
total_accidents = df_since_july_2012.shape[0]

# Calculate the average number of accidents with at least one fatality per 1000 accidents
average_fatal_accidents_per_1000 = (total_accidents_at_least_one_fatality / total_accidents) * 1000

average_fatal_accidents_per_1000

1.4286660513652585

In [12]:
missing_cross_street_count = df_since_july_2012['CROSS STREET NAME'].isna().sum()

# Calculate the proportion of accidents without a Cross Street Name
proportion_missing_cross_street = missing_cross_street_count / total_accidents

proportion_missing_cross_street

0.37751303808857534

In [13]:
vehicle_combination_counts = df_since_july_2012.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size().sort_values(ascending=False)

# Get the combination with the most number of accidents
most_common_vehicle_combination = vehicle_combination_counts.index[0]
most_common_vehicle_combination_count = vehicle_combination_counts.iloc[0]

most_common_vehicle_combination, most_common_vehicle_combination_count

(('Sedan', 'Sedan'), 204861)

In [14]:
# Filter the data for accidents with alcohol involvement as the contributing factor
df_alcohol_involved = df_since_july_2012[df_since_july_2012['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Count the number of accidents with alcohol involvement and at least one fatality
alcohol_involved_fatalities = df_alcohol_involved[df_alcohol_involved['NUMBER OF PERSONS KILLED'] >= 1].shape[0]

# Calculate the proportion of alcohol-involved accidents that resulted in a fatality
proportion_alcohol_involved_fatalities = alcohol_involved_fatalities / df_alcohol_involved.shape[0]

proportion_alcohol_involved_fatalities


0.004648897547153104

In [15]:
# Extract hour from 'CRASH TIME' and create a new column
df_since_july_2012['CRASH HOUR'] = pd.to_datetime(df_since_july_2012['CRASH TIME']).dt.hour

# Filter the data for accidents occurring between 4 PM and 6:59 PM (inclusive)
df_evening_rush_hour = df_since_july_2012[(df_since_july_2012['CRASH HOUR'] >= 16) & (df_since_july_2012['CRASH HOUR'] < 19)]

# Calculate the proportion of accidents occurring during the evening rush hour
proportion_evening_rush_hour = df_evening_rush_hour.shape[0] / total_accidents

proportion_evening_rush_hour

0.20445879180782098

In [16]:
# Filter the data for accidents involving motorcycles
df_motorcycle_involved = df_since_july_2012[(df_since_july_2012['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE', case=False, na=False)) | 
                                            (df_since_july_2012['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE', case=False, na=False))]

# Count the number of motorcycle-involved accidents with injuries but no fatalities
motorcycle_injuries_no_fatalities = df_motorcycle_involved[(df_motorcycle_involved['NUMBER OF PERSONS INJURED'] > 0) & 
                                                           (df_motorcycle_involved['NUMBER OF PERSONS KILLED'] == 0)].shape[0]

# Calculate the proportion of motorcycle-involved accidents resulting in injuries but no fatalities
proportion_motorcycle_injuries_no_fatalities = motorcycle_injuries_no_fatalities / df_motorcycle_involved.shape[0]

proportion_motorcycle_injuries_no_fatalities


0.5635715876196838

In [18]:
# Count the number of accidents involving bicycles
bicycle_involved_count = df_since_july_2012[(df_since_july_2012['VEHICLE TYPE CODE 1'].str.contains('BICYCLE', case=False, na=False)) | 
                                            (df_since_july_2012['VEHICLE TYPE CODE 2'].str.contains('BICYCLE', case=False, na=False))].shape[0]

bicycle_involved_count


19108

In [19]:
# Count the number of accidents with missing 'CROSS STREET NAME'
missing_cross_street_count = df_since_july_2012['CROSS STREET NAME'].isna().sum()

# Calculate the proportion of accidents without a Cross Street Name
proportion_missing_cross_street = missing_cross_street_count / total_accidents

proportion_missing_cross_street


0.37751303808857534